# 1b. Backpropagation

So I spent the past week and a half convincing myself that I understand the math behind backpropagation, and now I'm going to first prove it all over again here, and then implement it for my baby neural net, so that I can start training it. 

I'm following along with [Nielsen Chapter 2](http://neuralnetworksanddeeplearning.com/chap2.html), which I really can't praise highly enough - if you don't currently have access to an ML professor who can walk you through the math in person, this chapter is a good substitute. Now that I've gone through it several times and written out my own proofs of various parts, I'm going to try to explain/implement this without referring back to the text.

To start off, let's define some notation.
* A particular layer of a neural net will be denoted as $l$. $l-1$, $l+1$, etc. refer to the layers before or after layer $l$, and $L$ denotes the final (output) layer.
* Assuming we've passed some value through the net, $a_j^l$ is the activation value of the $j$th neuron in layer $l$ (the output of the activation function). $z_j^l$ is the pre-activation value, that is, the weighted sum of the inputs from the previous layer plus the bias (so $a_j^l = \sigma(z_j^l)$ where $\sigma$ is that neuron's activation function. We can also refer to $a^l$, $b^l$, or $z^l$ to denote the vector of the given quantities for layer $l$ (e.g. $a^l = [a_0^l, ..., a_n^l]$, where layer $l$ has $n$ neurons total).
* $w_{jk}^l$ is the current weight from neuron $k$ in layer $l-1$ to neuron $j$ in layer $l$ (i.e. $w_{jk}^l$ is the weight *to* $j$ *from* $k$. (This is backwards for reasons that make more sense when you think of a neural net as a list of weight matrices and bias vectors,  but we'll use this convention to abide by historical precedent.) $b_j^l$ is the current bias for neuron $j$ in layer $l$.

To train a neural net, we have some set of input values that are paired with the corresponding desired output values. We also have a cost function of some kind that provides a measure of the difference between the desired output and the actual output of the neural net.

* If our net has $n$ neurons in the input layer and $m$ in the output layer, then $(x, y)$ is a training example, where $x$ is a vector of length $n$ and $y$ is a vector of length $m$.
* If we feed a training input $x$ through the network, we get some output vector $a^L$.

Our set of training examples is fixed; we're trying to change all the weights and biases in the network so that overall, $a^L$ for each $x$ is pretty close to the corresponding $y$. So at a high level, think of the cost function as something that takes a bunch of candidate values for each weight and bias, applies those weights and biases to each training example, and outputs some measure of the cumulative cost. We want to find some particular set of weights and biases that minimizes the cumulative cost. Our cost function is not a function $C(y, a^L)$ for particular examples; it's more like a function $C_T(w^L, w^{L-1}, ..., w^0, b^L, b^{L-1}, ..., b^0)$, parameterized by the training set $T$ and varying over the weights and biases. This is why it makes sense to talk about minimizing the cost by changing them - it's a function of those quantities and we can do calculus to it.

The overall cost function $C$ measures some cumulative error over a bunch of training examples. For backpropagation, though, we want to be able to see how the net processes individual examples, so we can figure out how the net needs to change to better suit each one. So we want our overall cost function to be easily decomposable into a cost function for a particular example - call this $C_x$. So without explicitly defining our cost function, we're going to impose a few restrictions on it:

* $C_T(w, b) = \frac{\epsilon}{|{T}|}\sum_{(x, y) \in T}C_x(w, b)$, that is, for a particular combo of weights and biases, $C_T$ is the average of individual errors $C_x$ over all pairs in the training set, maybe scaled by some constant $\epsilon$ depending on the exact form.
* $C_x(w, b)$ should be easily differentiable. We want to minimize $C_T$, which means we're going to be taking derivatives with respect to each weight and bias, and in this form the derivative of  $C_T$ works out to the (scaled) average of the derivatives of all the individual $C_x$s.

$\frac{\partial C_T}{\partial w_{jk}^l}$ (or $\frac{\partial C_T}{\partial b_j^l}$)